In [15]:
max_len = 50

In [18]:
use_gpu = True
DEVICE = "cuda" if use_gpu else "cpu"
DEVICE

'cuda'

In [19]:
q1 = "How can I lose weight?"
q2 = "What is the best way to reduce body fat?"

In [20]:
with open("../registry/vocabularies/vocab.json") as f:
    vocab = json.load(f)

embedding_matrix = torch.load("../registry/embeddings/embedding_matrix.pt")

model = QuoraDuplicateGRU(embedding_matrix, hidden_size=100)
model.load_state_dict(torch.load("../registry/models/gru/gru_model.pt"))
model.eval()


QuoraDuplicateGRU(
  (embedding): Embedding(50000, 300, padding_idx=0)
  (q_encoder): GRU(300, 100, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=800, out_features=1, bias=True)
)

In [21]:
def text_to_sequence(text, vocab, max_len=40):
    tokens = text.lower().split()
    seq = [vocab.get(token, vocab.get("<UNK>", 0)) for token in tokens]
    if len(seq) < max_len:
        seq += [vocab["<PAD>"]] * (max_len - len(seq))
    return seq[:max_len]

q1_tensor = torch.tensor([text_to_sequence(q1, vocab, max_len)[0]], dtype=torch.long).to(DEVICE)
q2_tensor = torch.tensor([text_to_sequence(q2, vocab, max_len)[0]], dtype=torch.long).to(DEVICE)

In [22]:
from captum.attr import IntegratedGradients

ig = IntegratedGradients(model)
attributions, delta = ig.attribute(
    inputs=(q1_tensor, q2_tensor),
    target=0,
    return_convergence_delta=True
)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)